# MHD

Simulate axisymmetric mhd.

In [ ]:
import dedalus.public as de
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
import file_tools as flt

In [ ]:
%env OMP_NUM_THREADS

In [ ]:
%env NUMEXPR_MAX_THREADS

# 2D $u_r, u_\theta, u_z$ in $(r,\theta)$

In [ ]:
sim_name_base = 'cylinder_obstacle_tracer'
save_dir = 'data/HeleShaw'
save_freq = 200#200
dt = 5e-5
timestepper = 'RK443'

In [ ]:
nr = 384
nθ = 2*384
χ = 0.2571428571428571 #0.428571 #0.257
γ = 0.02249718785151856 #0.0225
ReC = 0.09210263786404081 #0.1
D = 0.24746906636670415#(1+χ)/8
κ = 2e-3 # diffusivity of dye
η = 1e-2 * np.min((ReC,1)) #1e-4 # damping timescale for volume penalty

θbasis = de.Fourier('θ',nθ,interval=(0,2*np.pi))
rbasis = de.Chebyshev('r',nr,interval=(χ,1))
domain = de.Domain([θbasis,rbasis],grid_dtype=np.float64)
θ, r = domain.grids()

simtime = dt*400#1/np.pi

In [ ]:
timescales_message = f'CFL number, dt/τ_CFL: {dt/((1-χ)/nr):.2E}, '
timescales_message += f'dt/η: {dt/η:.2E}'
print(timescales_message)

In [ ]:
sim_name = sim_name_base+'-χ='+str(np.round(χ,2)).replace('.','_')+'-γ='+str(np.round(γ,3)).replace('.','_')+'-ReC='+str(np.round(ReC,3)).replace('.','_')+'-D='+str(np.round(D,3)).replace('.','_')
sim_name

In [ ]:
x = r*np.cos(θ)
y = r*np.sin(θ)
x0 = (1+χ)/2
y0 = 0
r0 = ((x-x0)**2+(y-y0)**2)**(1/2)
mask = lambda x: 0.5*(1+np.tanh(-2*x))
ϵ = γ*(η/ReC)**(1/2)
δ = 2.64822828*ϵ # sharpness of mask for obstacle
K = domain.new_field() # mask
K['g'] = mask((r0-D/2)/δ)

fig, ax = plt.subplots()
ax.pcolormesh(x,y,K['g'],shading='gouraud',cmap='Reds',vmin=0,vmax=1)
ax.set(aspect=1)
plt.show()

In [ ]:
problem = de.IVP(domain,variables=['p','ur','urr','uθ','uθr','c','cr'])

problem.parameters['χ'] = χ
problem.parameters['γ'] = γ
problem.parameters['ReC'] = ReC
problem.parameters['κ'] = κ
problem.parameters['η'] = η
problem.parameters['K'] = K
problem.parameters['δ'] = δ
problem.parameters['I'] = 1

problem.substitutions['l'] = 'log(1-K+δ)'
problem.substitutions['q'] = 'uθr - dθ(ur)/r + uθ/r'

problem.add_equation("ur + dθ(uθ) + r*dr(ur) = 0")
problem.add_equation("dr(ur) - urr = 0")
problem.add_equation("dr(uθ) - uθr = 0")
problem.add_equation("dr(c) - cr = 0")
problem.add_equation("r**2*dt(uθ) + r*(1 + χ)*dθ(p) - 1/ReC*γ**2*(dθ(dθ(uθ) + 2*ur) + r*dr(r*uθr)) + 1/ReC*(2*r**2 + γ**2)*uθ = (1 + χ)*r*(-uθ*ur - uθ*dθ(uθ) - r*ur*uθr + I/ReC) - r**2*(K*uθ)/η")
problem.add_equation("r**2*(dt(ur) + (1 + χ)*dr(p)) - 1/ReC*γ**2*(dθ(dθ(ur) - 2*uθ) + r*dr(r*urr)) + 1/ReC*(2*r**2 + γ**2)*ur = (1 + χ)*r*(uθ**2 - uθ*dθ(ur) - r*ur*urr) - r**2*(K*ur)/η")
problem.add_equation("r**2*(dt(c) - κ*(dθ(dθ(c)) + r*dr(r*cr))) = -r*uθ*dθ(c)-r**2*ur*cr + dθ(l)*(r*c*uθ-κ*dθ(c)) + r**2*dr(l)*(c*ur-κ*cr)")

problem.add_equation('left(uθ) = 0')
problem.add_equation('left(ur) = 0')
problem.add_equation('right(uθ) = 0')
problem.add_equation('right(ur) = 0',condition='nθ!=0')
problem.add_equation('right(p) = 0',condition='nθ==0')
problem.add_equation('right(urr) = 0')
problem.add_equation('left(cr) = 0')
problem.add_equation('right(cr) = 0')


solver = problem.build_solver(getattr(de.timesteppers, timestepper))

In [ ]:
p, ur, urr, uθ, uθr,c,cr = [solver.state[_] for _ in problem.variables]
#impulse_func.original_args = impulse_func.args = [solver]

flt.makedir(save_dir)
analysis = solver.evaluator.add_file_handler(f'{save_dir}/analysis-{sim_name}',iter=save_freq)
for f in problem.variables + ['q']: analysis.add_task(f)

In [ ]:
# Specify initial dye pattern
bump = lambda x: np.piecewise(x, [np.abs(x)<1, np.abs(x)>=1], 
                              [lambda x: np.exp(1)*np.exp(-1 / (1 - x**2)), lambda x: 0])
streak = lambda x, x0: bump((x-x0)/streak_half_width)
# streak = lambda x, x0: mask((x-x0-streak_half_width )/δ)*mask(-(x-x0+streak_half_width)/δ)

dye_comments ='zonal streaks'

if dye_comments == 'radial streaks':
    streak_half_width = 7*(2*np.pi/nθ)
    dye_pattern = lambda θ : sum(streak(θ,θi) for θi in (np.linspace(0,2*np.pi,8)+0.17)) * (1- mask((r0-D/2)/δ))
    pattern_domain = θ
if dye_comments == 'zonal streaks':
    streak_half_width = 7*((1-χ)/nr)
    streaks_per_obst = 4
    spacing = np.max((D/(streaks_per_obst-1),3/2*streak_half_width))
    dye_pattern = lambda r : sum(streak(r,ri) for ri in ((1-χ)/2*np.linspace(-1,1,int((1-χ)/spacing ))+(1+χ)/2)) * (1- mask((r0-D/2)/δ))
    pattern_domain = r

c['g'] = dye_pattern(pattern_domain)
cr['g'] = c.differentiate('r')['g']

In [ ]:
theta_cyl = np.linspace(0,2*np.pi,100)
x_cyl_o, y_cyl_o = np.cos(theta_cyl),np.sin(theta_cyl)
x_cyl_i,y_cyl_i = χ*np.cos(theta_cyl),χ*np.sin(theta_cyl)
x_obs, y_obs = D/2*np.cos(theta_cyl)+(χ+1)/2,D/2*np.sin(theta_cyl)

fig, ax = plt.subplots(1,2,figsize=(10,5))
ax[0].pcolormesh(x,y,c['g'],shading='gouraud',cmap='Blues',vmin=[0,1])
ax[0].contour(x,y,K['g'],[0.5],colors='r',linewidths=1)
ax[0].plot(x_obs, y_obs,color='k')
ax[0].set(aspect=1)

#ax[1].plot(np.linspace(0,2*np.pi,10000),streak(np.linspace(0,2*np.pi,10000),np.pi))
#ax[1].scatter(np.linspace(0,2*np.pi,nθ),streak(np.linspace(0,2*np.pi,nθ),np.pi),color='r')
ax[1].scatter(r[0,:],c['g'][int(nθ/2),:])
#ax[1].set_ylim(0,1.5)
#ax[1].set_xlim(0.75*np.pi,1.25*np.pi)

In [ ]:
%%time
while solver.sim_time <= simtime:
    solver.step(dt)
    if solver.iteration % 100 == 0: 
        print(f'it {solver.iteration}',
              f't {solver.sim_time:.3f}',
              f'|uθ| max {np.abs(uθ["g"]).max():.3f}')
        if np.any(np.isnan(uθ['g'])): 
            print('Broken')
            break

## Analysis

In [ ]:
import glob
from dedalus.tools import post

In [ ]:
file_dir = f'{save_dir}/analysis-{sim_name}'
post.merge_analysis(file_dir,cleanup=False)
save_file = sorted(glob.glob(f'{file_dir}/*.h5'))[0]

In [ ]:
data = {}

In [ ]:
data['t'],data['r'],data['θ'] = flt.load_data(save_file,'sim_time','r/1.0','θ/1.0',group='scales')
for key in flt.get_keys(save_file,group='tasks'):
    data[key], = flt.load_data(save_file,key,group='tasks')

In [ ]:
data['x'], data['y'] = data['r'][None,:]*np.cos(data['θ'][:,None]), data['r'][None,:]*np.sin(data['θ'][:,None])

In [ ]:
data['c'][0]

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(data['x'], data['y'], 
               data['c'][-1],
               shading='gouraud',cmap='Blues',vmin=[0,0.3])
#ax.contour(data['x'], data['y'],K['g'],[.5],colors='k')
ax.set(aspect=1)

In [ ]:
fig, ax = plt.subplots()
ax.pcolormesh(data['x'], data['y'], 
               data['q'][-1],
               shading='gouraud',cmap='PuOr_r',vmin=[-1,1])
#ax.contour(data['x'], data['y'],K['g'],[.5],colors='k')
ax.set(aspect=1)

In [ ]:
data['t']